In [17]:
import os
from openai import OpenAI
from sqlalchemy import create_engine, MetaData, Table, select, not_, text

# Database setup for MS SQL Server (Integrated Security)
SERVER = '(localdb)\\MSSQLLocalDB'
DATABASE = 'GroceryDB'
DRIVER = 'ODBC Driver 17 for SQL Server'

# Correct SQLAlchemy URL
DATABASE_URL = f"mssql+pyodbc://{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}&trusted_connection=yes"
engine = create_engine(DATABASE_URL)
metadata = MetaData()

# Define tables
Recipes = Table("Recipes", metadata, autoload_with=engine)
MajorIngredientFromRecipeName = Table("MajorIngredientFromRecipeName", metadata, autoload_with=engine)

# OpenAI client setup
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # New client initialization

def get_chatgpt_ingredients(recipe_name: str) -> list[str]:
    """Extract major ingredients using ChatGPT (updated for new API)."""
    response = client.chat.completions.create(  # Updated API call
        model="gpt-3.5-turbo",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant that extracts major ingredients from recipe names. Always output a bulleted list of ingredients in singular form and without explanation."},
            {
            "role": "user",
            "content": f"""
                Extract the major ingredients from the following recipe name. 
                Return only the ingredients as a bulleted list in singular form, with no explanation.
                Recipe name: "{recipe_name}"
            """
        }],
        temperature=0.1,
        max_tokens=200
    )
    ingredients_text = response.choices[0].message.content.strip()  # Updated attribute access
    return [
        line.replace("-", "").strip().lower() 
        for line in ingredients_text.split("\n") 
        if line.strip()
    ]
    

def process_recipes():
    """Fetch 5 unprocessed recipes, call ChatGPT, and save ingredients."""
    with engine.connect() as conn:
        # Get 5 RecipeIds not already processed (using text() for raw SQL)
        query = text(f"""
            SELECT TOP 5 RecipeId, name 
            FROM Recipes 
            WHERE RecipeId NOT IN (
                SELECT DISTINCT RecipeId FROM MajorIngredientFromRecipeName
            )
        """)
        recipes = conn.execute(query).fetchall()

        print(f"Found {len(recipes)} recipes to process. Starting...")
        for recipe_id, recipe_name in recipes:
            try:
                print(f"\nProcessing RecipeId {recipe_id}: '{recipe_name}'...")
                ingredients = get_chatgpt_ingredients(recipe_name)
                print("Extracted ingredients:", ingredients)

                for ingredient in ingredients:
                    conn.execute(
                        text(f"""
                            INSERT INTO MajorIngredientFromRecipeName (RecipeId, IngredientName)
                            VALUES (:recipe_id, :ingredient)
                        """), 
                        {"recipe_id": recipe_id, "ingredient": ingredient}
                    )
                conn.commit()
                print("✅ Saved to database.")
            except Exception as e:
                print(f"❌ Error: {e}")
                conn.rollback()

if __name__ == "__main__":
    process_recipes()
    # print(get_chatgpt_ingredients("Chocolate Chip Cookies"))
    print("\nTest run completed. Check the database for results.")

Found 5 recipes to process. Starting...

Processing RecipeId 10008: 'Roasted Pork Jowl And Feet Porridge'...
Extracted ingredients: ['pork jowl', 'pork feet', 'porridge']
✅ Saved to database.

Processing RecipeId 10010: 'Ginger Chicken Soup'...
Extracted ingredients: ['ginger', 'chicken', 'soup']
✅ Saved to database.

Processing RecipeId 10013: 'Stir-Fried Bok Choy and Lettuce with Mushrooms'...
Extracted ingredients: ['bok choy', 'lettuce', 'mushrooms']
✅ Saved to database.

Processing RecipeId 10014: 'Rice Noodles with Tofu and Bok Choy'...
Extracted ingredients: ['rice noodle', 'tofu', 'bok choy']
✅ Saved to database.

Processing RecipeId 10017: 'Hoisin Tofu and Vegetables'...
Extracted ingredients: ['hoisin', 'tofu', 'vegetables']
✅ Saved to database.

Test run completed. Check the database for results.
